<a href="https://colab.research.google.com/github/Chabon6/Automated_Ranking_Checker_Bot/blob/main/DtatAnalyst%26ManagerAssistant_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet openpyxl
!pip install --upgrade --quiet langchain langchain-openai langchain_community langchain_experimental

In [2]:
import pandas as pd
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
from google.colab import userdata
gpt_api = userdata.get('gpt_api')

In [3]:
# prompt: 讀取googledrive的檔案

from google.colab import drive
drive.mount('/content/drive')

# Replace 'your_file_path' with the actual path to your file in Google Drive
file_path = '/content/drive/MyDrive/LangChain練習/銀行客戶流失.xlsx'

df = pd.read_excel(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   CustomerId   Surname  CreditScore  Geography  Gender  Age    Balance  \
0    15634602  Hargrave          619     Taiwan  Female   42       0.00   
1    15647311      Hill          608  Singapore  Female   41   83807.86   
2    15619304      Onio          502     Taiwan  Female   42  159660.80   
3    15701354      Boni          699     Taiwan  Female   39       0.00   
4    15737888  Mitchell          850  Singapore  Female   43  125510.82   

   NumOfProducts  IsActiveMember  EstimatedSalary  Exited  Point Earned  \
0              1               1        101348.88       1           464   
1              1               1        112542.58       0           456   
2              3               0        113931.57       1           377   
3              2               0         93826.63       0           350   
4              1               1         7908

In [4]:
# 設定語言模型
analyst_llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_retries=2,
    api_key=gpt_api
)

manager_llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_retries=2,
    api_key=gpt_api
)

<ipython-input-4-cd4f9d0067db>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  analyst_llm = ChatOpenAI(


In [21]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 定義分析師跟主管的 prompt
prompt_analyst = PromptTemplate(
                  input_variables=["Quation"],
                  template='''
                  <rule>你是一位專業的資料分析師，負責分析數據並產出有用的商業洞察及發現，必要時可以自創比率，以協助主管達成商業決策或解決商業問題。注意：必須以數據或比率支持你的觀點，</rule>
                  \n\n
                  <format>*若需要以圖表說明，請以字串表達python的繪圖程式碼
                      *Markdown格式回答
                      *列點式回答
                      *回答須包含：1.發現、2.可能原因、3.什麼數據顯示這種可能原因</format>
                  <task>主管要求：{Quation}</task>
                  '''
                  )
prompt_manager = PromptTemplate(
                  input_variables=["analyst_response"],
                  template='''
                  <rule>你是一位專業的商業戰略主管。負責判斷資料分析師所提供的分析結果，來決定是否請分析師進一步分析。</rule>
                  <background>資料分析師根據你最一開始的提問：{initial_question}做了一份分析結果，你規定結果必須包含以下三點：1.發現、2.可能原因、3.什麼數據顯示這種可能原因</background>
                  <task>
                  仔細審視資料分析師給你的分析結果，並判斷分析結果是否符合你規定的必要三點：1.發現、2.可能原因、3.什麼數據顯示這種可能原因。
                    if: 你覺得分析師提供的分析結果已經足夠，可以直接回答"ok到此為止"，無須其他廢話。
                    else: 你覺得分析結果未能滿足你，請提出更深入的問題及要求，以挖掘更多潛在的商業洞察、商機或風險。
                  </task>
                  <AnalysisResults>資料分析師提供的分析結果：{analyst_response}</AnalysisResults>
                  '''
                  )

# 創建分析師跟主管的 LLM Chain
# 設定agent
from langchain_experimental.agents import create_pandas_dataframe_agent
agent_executor = create_pandas_dataframe_agent(
    analyst_llm,
    df,
    verbose=True,
    agent_type="openai-functions",
    allow_dangerous_code=True
)
manager_chain = LLMChain(llm=manager_llm, prompt=prompt_manager)

In [22]:
initial_question = "該如何降低客戶的流失率？"
# 創建對話存放表
data = {
"分析師": [],
"主管": []
}
dt_respone = pd.DataFrame(data)

# 第一次分析
analyst_response = agent_executor.invoke({"input":prompt_analyst.format(Quation=initial_question)}) # 初始分析
analyst_response_text = analyst_response['output']
dt_respone.loc[0, "分析師"] = analyst_response_text



> Entering new AgentExecutor chain...
### 降低客戶流失率的建議

1. **發現：高信用分數的客戶流失率較低**
   - **可能原因**：高信用分數的客戶可能更有財務穩定性，對銀行的服務需求更高。
   - **數據支持**：可以計算不同信用分數區間的客戶流失率，並比較其差異。

   ```python
   import pandas as pd

   # 計算不同信用分數區間的客戶流失率
   df['CreditScoreBin'] = pd.cut(df['CreditScore'], bins=[300, 500, 700, 850], labels=['Low', 'Medium', 'High'])
   credit_score_exit_rate = df.groupby('CreditScoreBin')['Exited'].mean()
   print(credit_score_exit_rate)
   ```

2. **發現：活躍會員的流失率較低**
   - **可能原因**：活躍會員可能對銀行的服務更滿意，或享受更多的會員福利。
   - **數據支持**：比較活躍會員和非活躍會員的流失率。

   ```python
   # 計算活躍會員和非活躍會員的流失率
   active_member_exit_rate = df.groupby('IsActiveMember')['Exited'].mean()
   print(active_member_exit_rate)
   ```

3. **發現：持有多種產品的客戶流失率較低**
   - **可能原因**：持有多種產品的客戶對銀行的依賴性更高，轉移成本較大。
   - **數據支持**：比較持有不同數量產品的客戶流失率。

   ```python
   # 計算持有不同數量產品的客戶流失率
   product_exit_rate = df.groupby('NumOfProducts')['Exited'].mean()
   print(product_exit_rate)
   ```

4. **發現：不同地區的客戶流失率存在差異**
   - **可能原因**：不同地區的市場競爭程度和客戶需求不同。
  

<ipython-input-22-8edb213cebf2>:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '### 降低客戶流失率的建議

1. **發現：高信用分數的客戶流失率較低**
   - **可能原因**：高信用分數的客戶可能更有財務穩定性，對銀行的服務需求更高。
   - **數據支持**：可以計算不同信用分數區間的客戶流失率，並比較其差異。

   ```python
   import pandas as pd

   # 計算不同信用分數區間的客戶流失率
   df['CreditScoreBin'] = pd.cut(df['CreditScore'], bins=[300, 500, 700, 850], labels=['Low', 'Medium', 'High'])
   credit_score_exit_rate = df.groupby('CreditScoreBin')['Exited'].mean()
   print(credit_score_exit_rate)
   ```

2. **發現：活躍會員的流失率較低**
   - **可能原因**：活躍會員可能對銀行的服務更滿意，或享受更多的會員福利。
   - **數據支持**：比較活躍會員和非活躍會員的流失率。

   ```python
   # 計算活躍會員和非活躍會員的流失率
   active_member_exit_rate = df.groupby('IsActiveMember')['Exited'].mean()
   print(active_member_exit_rate)
   ```

3. **發現：持有多種產品的客戶流失率較低**
   - **可能原因**：持有多種產品的客戶對銀行的依賴性更高，轉移成本較大。
   - **數據支持**：比較持有不同數量產品的客戶流失率。

   ```python
   # 計算持有不同數量產品的客戶流失率
   product_exit_rate = df.groupby('NumOfProducts')[

In [23]:
for i in range(3):
    manager_question = manager_chain.invoke({
                          "analyst_response": analyst_response_text,
                          "initial_question": initial_question
                          },
                          handle_parsing_errors=True)
    manager_question_text = manager_question["text"]
    print(f"主管：{manager_question_text}")
    dt_respone.loc[i, "主管"] = manager_question_text
    if "ok到此為止" in manager_question_text.lower():
        break

    analyst_response = agent_executor.invoke({"input":prompt_analyst.format(Quation=manager_question_text)},handle_parsing_errors=True)
    analyst_response_text = analyst_response['output']
    print(f"分析師：{analyst_response_text}")
    dt_respone.loc[i+1, "分析師"] = analyst_response_text

print(dt_respone)

主管：請問您對資料分析師提供的分析結果有何看法？是否符合您的期望？您是否需要更深入的分析或資訊來做出決策？


> Entering new AgentExecutor chain...


<ipython-input-23-5c3e80ae5115>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '請問您對資料分析師提供的分析結果有何看法？是否符合您的期望？您是否需要更深入的分析或資訊來做出決策？' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dt_respone.loc[i, "主管"] = manager_question_text


### 分析結果

1. **發現**
   - 客戶流失率（Exited = 1）似乎與信用評分（CreditScore）和餘額（Balance）有一定關聯。
   - 客戶的年齡（Age）和是否為活躍會員（IsActiveMember）也可能影響流失率。
   - 不同地區（Geography）的客戶流失率可能存在差異。

2. **可能原因**
   - **信用評分低的客戶流失率較高**：信用評分較低的客戶可能更容易遇到財務困難，從而選擇離開。
   - **餘額為零的客戶流失率較高**：餘額為零的客戶可能對銀行的依賴性較低，從而更容易流失。
   - **年齡較大的客戶流失率較高**：年齡較大的客戶可能面臨退休或其他生活變動，從而影響其與銀行的關係。
   - **非活躍會員流失率較高**：非活躍會員可能對銀行的產品和服務不滿意，從而選擇離開。
   - **地區差異**：不同地區的經濟環境和競爭情況可能影響客戶的流失率。

3. **什麼數據顯示這種可能原因**
   - **信用評分與流失率的關聯**：可以通過計算不同信用評分範圍內的客戶流失率來驗證。
   - **餘額與流失率的關聯**：可以通過計算不同餘額範圍內的客戶流失率來驗證。
   - **年齡與流失率的關聯**：可以通過計算不同年齡範圍內的客戶流失率來驗證。
   - **活躍會員與流失率的關聯**：可以通過計算活躍會員和非活躍會員的流失率來驗證。
   - **地區差異**：可以通過計算不同地區的客戶流失率來驗證。

### 進一步分析建議

1. **信用評分與流失率的關聯分析**
   ```python
   import matplotlib.pyplot as plt
   import seaborn as sns

   plt.figure(figsize=(10, 6))
   sns.histplot(data=df, x='CreditScore', hue='Exited', multiple='stack', bins=30)
   plt.title('Credit Score Distribution by Exited Status')
   plt.xlabel('Credit Score')
   plt.ylabel('Count')
   plt.

In [24]:
dt_respone

,分析師,主管
0,### 降低客戶流失率的建議\n\n1. **發現：高信用分數的客戶流失率較低**\n ...,請問您對資料分析師提供的分析結果有何看法？是否符合您的期望？您是否需要更深入的分析或資訊來做...
1,### 分析結果\n\n1. **發現**\n - 客戶流失率（Exited = 1）似...,ok到此為止


# 等等

In [ ]:
# 創建對話循環
def dialogue_loop(df, initial_question):
    # 創建對話存放表
    data = {
    "分析師": [],
    "主管": []
    }
    dt_respone = pd.DataFrame(data)

    # 第一次分析
    analyst_response = agent_executor.invoke({"input":prompt_analyst.format(Quation=initial_question)}) # 初始分析
    analyst_response_text = analyst_response['output']
    dt_respone.loc[0, "分析師"] = analyst_response_text

    for i in range(3):
        manager_question = manager_chain.invoke({"analyst_response": analyst_response_text},handle_parsing_errors=True)
        print(f"主管：{manager_question}")
        dt_respone.loc[i, "主管"] = manager_question
        if "ok到此為止" in analyst_response_text.lower():
            break

        analyst_response = agent_executor.invoke({"input":prompt_analyst.format(Quation=manager_question)},handle_parsing_errors=True)
        print(f"分析師：{analyst_response_text}")
        analyst_response_text = analyst_response['output']
        dt_respone.loc[i+1, "分析師"] = analyst_response_text

    print(dt_respone)

# 開始對話
dialogue_loop(df, "該如何降低客戶的流失率？")